# Ingest NCEP GFS 0.25 Degree Data for 6 hour forecasts. 

#### 1.) Conda package installations to environment and importing appropriate libraries. 

In [1]:
# conda install -c conda-forge gdal
# conda install -c conda-forge geopandas
# conda install -c conda-forge earthpy
# conda install -c conda-forge cloudpathlib
# conda install -c conda-forge pyhdf
# conda install -c anaconda basemap

#conda install -c conda-forge ipywidgets
#conda install -c conda-forge cartopy

## For IO dependencies in xarray 
#conda install -c conda-forge xarray dask netCDF4 bottleneck
#conda install -c conda-forge cfgrib
#conda install -c conda-forge pygrib
#conda install -c yt87 pywgrib2_xr

## For writing / reading parquet files 
#conda install -c conda-forge pyarrow

#For s3 
#conda install -c conda-forge boto3

#For timing 
#conda install -c conda-forge profilehooks

In [2]:
#Import Packages. 
import sys
import os
import requests
import warnings
import glob
import time

import matplotlib.pyplot as plt
import seaborn as sns
import numpy.ma as ma
import numpy as np
#from shapely.geometry import mapping, box
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from osgeo import gdal
import pandas as pd

#GFS data
import xarray # used for reading the data.
import xarray_extras.csv # used for writing data to csv format. 
import pygrib
import xarray # used for reading the data.
import ipywidgets as widgets
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.

#For writing to s3
import boto3
import io
import pickle

#For timing function
from profilehooks import profile

#For multiprocessing of function
import multiprocessing
from multiprocessing import Pool

# #from cloudpathlib import S3Path, S3Client
# from pyhdf.SD import SD, SDC

warnings.simplefilter('ignore')

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
#Get number of CPUs
print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  4


In [4]:
#Define lat/lon bounds of our regions of interest. 
#Note: We must convert the original lon bounds of -180, 180 --> 0, 360 to match the GFS data format. 

#https://docs.google.com/spreadsheets/d/1sZwAc0dSuYTlZNeZEEu0320iUIQPO-WBjSGu50nlFpE/edit?resourcekey=0-4yL7WtTJDHthV8swuq7SBg#gid=0
#Los Angeles
la_min_lat = 30.01
la_max_lat = 40.00
la_min_lon = 49.46
la_max_lon = 76.06
la_bounds = [la_min_lat, la_max_lat, la_min_lon, la_max_lon]

#Tapei
tp_min_lat = 20.01
tp_max_lat = 30.00
tp_min_lon = 297.07
tp_max_lon = 318.55
tp_bounds = [tp_min_lat, tp_max_lat, tp_min_lon, tp_max_lon]

#Delhi
dl_min_lat = 20.01
dl_max_lat = 30.00
dl_min_lon = 243.85
dl_max_lon = 260.82
dl_bounds = [dl_min_lat, dl_max_lat, dl_min_lon, dl_max_lon]

In [5]:
#Filter by appropriate lat/lon bounds
def subset_dataset(dataset, min_lat, max_lat, min_lon, max_lon): 
    '''Takes a dataset and bounding coordinates and returns a filtered subset for the region of interest'''
    mask_lat = np.logical_and(dataset.coords['latitude'] >= min_lat, dataset.coords['latitude'] <= max_lat)
    mask_lon = np.logical_and(dataset.coords['longitude'] >= min_lon, dataset.coords['longitude'] <= max_lon)
    ds_filt = dataset.where(mask_lat & mask_lon, drop = True)
    return ds_filt

#### 2.) Download data from NCAR servers. 

In [6]:
 ## First, we need to authenticate
try:
    import getpass
    input = getpass.getpass
except:
    try:
        input = raw_input
    except:
        pass

In [7]:
## Now, we need your password.
pswd = input('password: ')

password:  ···········


In [8]:
values = {'email' : 'jericojohns@berkeley.edu', 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

In [9]:
ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

In [10]:
dspath = 'https://rda.ucar.edu/data/ds084.1/'
save_dir = '/local/train/GFS/'
filelist = []

In [11]:
#@profile(immediate=True)
def run_gfs_pipeline(years, months, days, times): 
    '''Pipeline the GFS data for specified times and specified variables, at the specified levels.
       Output will be a parquet file with 1 row per unique lat/lon combination (within regions of interest). 
       Forecast times will be added column-wise so that there are 4 forecasts per variable per day per row.'''
    for year in years: 
        for month in months: 
            for day in days: 
                final_df = pd.DataFrame()
                for time in times:
                    #Download .glib2 file (temporarily) to scrape the desired fields. We will delete after use. 
                    file = year + '/' + year + month + day + '/gfs.0p25.' + year + month + day + time + '.f006.grib2'
                    filename = dspath + file
                    outfile = save_dir + os.path.basename(filename) 
                    print('Downloading', file)
                    req = requests.get(filename, cookies = ret.cookies, allow_redirects=True)
                    open(outfile, 'wb').write(req.content)
                    filelist_arr = [save_dir + os.path.basename(file)]
                    selected_file = widgets.Dropdown(options=filelist_arr, description='data file')
                    display(selected_file)

                    #Now we use xarray to open the file by the type_of_level we are interested in 
                    type_of_level1 = 'surface' # for Temperature and Planetary Boundary Layer Height
                    type_of_level2 = 'atmosphereSingleLayer' # for Relative Humidity
                    ds_level_surface = xarray.open_dataset(selected_file.value, filter_by_keys={'typeOfLevel': type_of_level1}, engine="cfgrib")
                    ds_level_atmosphere = xarray.open_dataset(selected_file.value, filter_by_keys={'typeOfLevel': type_of_level2}, engine="cfgrib")

                    #Define variable names
                    var_t = 't' #temperature (K) 
                    var_hpbl = 'hpbl' #Planetary Boundary Layer Height (m)
                    var_r = 'r' #Relative Humidity %

                    #Define filtered datasets (for each variable). 
                    ds_t = ds_level_surface[var_t] 
                    ds_hpbl = ds_level_surface[var_hpbl]
                    ds_r = ds_level_atmosphere[var_r]

                    #Initialize empty dataframe to append each regional dataframe to
                    daily_df = pd.DataFrame()
                    for min_lat, max_lat, min_lon, max_lon in [la_bounds, tp_bounds, dl_bounds]: 
                        #Filter to bounds of regions of interest. 
                        ds_t_filt = subset_dataset(ds_t, min_lat, max_lat, min_lon, max_lon)
                        ds_hpbl_filt = subset_dataset(ds_hpbl, min_lat, max_lat, min_lon, max_lon)
                        ds_r_filt = subset_dataset(ds_r, min_lat, max_lat, min_lon, max_lon)


                        #Make sure we preserve the type of level (atmospheric) of the observation to preserve metadata within the variable names
                        df_t = ds_t_filt.to_dataframe(name = var_t)
                        df_t = df_t.drop(columns = ['surface', 'time', 'step', 'valid_time'])
                        df_t = df_t.rename(columns = {"t" : "t_surface" + time, "hpbl" : "pbl_surface" + time, "r" : "r_atmosphere_single_layer" + time})

                        df_pbl = ds_hpbl_filt.to_dataframe(name = var_hpbl)
                        df_pbl = df_pbl.drop(columns = ['surface', 'time', 'step', 'valid_time'])
                        df_pbl = df_pbl.rename(columns = {"t" : "t_surface" + time, "hpbl" : "pbl_surface" + time, "r" : "r_atmosphere_single_layer" + time})

                        df_r = ds_r_filt.to_dataframe(name = var_r)
                        df_r = df_r.drop(columns = ['atmosphereSingleLayer', 'time', 'step', 'valid_time'])
                        df_r = df_r.rename(columns = {"t" : "t_surface" + time, "hpbl" : "pbl_surface" + time, "r" : "r_atmosphere_single_layer" + time})


                        #Now join all fields into same df
                        joined_df_current = pd.merge(df_t, df_pbl, on = ["latitude", "longitude"], how = "left")
                        joined_df_current = pd.merge(joined_df_current, df_r, on = ["latitude", "longitude"], how = "left")

                        #Now concatenate current dataframe into final dataframe
                        if daily_df.empty: 
                            daily_df = joined_df_current
                        else: 
                            daily_df = pd.concat([daily_df, joined_df_current], axis = 0)

                     #Now we delete the .grib2 file so as to save memory. Otherwise, we'd be storing ~1tb of data. 
                    if os.path.exists(outfile):
                        os.remove(outfile)
                    if os.path.exists(outfile + '.923a8.idx'):
                        os.remove(outfile + '.923a8.idx')
                            
                    #Join the different forecast time dataframes together so that we have one column per forecast time. 
                    if final_df.empty: 
                        final_df = daily_df
                    else: 
                        final_df = pd.merge(final_df, daily_df, on = ["latitude", "longitude"], how = "left")

                            
                #Convert to final_df to parquet, with the appropriate metadata in file name (will extract as field names later). 
                out_parquet =  'gfs.0p25.' + year + month + day + '.f006.parquet'
                
                #For now just upload to s3
                # filepath = '../train/GFS/parquet/' + out_parquet
                # final_df.to_parquet(path = filepath, engine = 'pyarrow')
                
                s3_write_client = boto3.client('s3')
                my_array_data = io.BytesIO()
                pickle.dump(final_df, my_array_data)
                my_array_data.seek(0)
                s3_write_client.upload_fileobj(my_array_data, 'pollution-prediction', out_parquet)

                        
    print("Pipeline complete.")

In [ ]:
%%time
#Now try to parellize the for loops 
years = ['2018']
months = ['01']
days = ['01', '02']
times = ['00', '06', '12', '18']

args = (years, months, days, times)

def pool_handler(): 
    p = Pool(4)
    p.map(run_gfs_pipeline, args)
    
if __name__ == '__main__':
    pool_handler()


In [12]:
%%time
#Iterate through all file names. 
# for year in ['2018', '2019', '2020']: 
#     for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']: 
#         for day in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']: 
#             for time in ['00', '06', '12', '18']: 


years = ['2018']
months = ['01']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']
times = ['00', '06', '12', '18']

run_gfs_pipeline(years, months, days, times)

Dropdown(description='data file', options=('/local/train/GFS/gfs.0p25.2018010100.f006.grib2',), value='/local/…

Ignoring index file '/local/train/GFS/gfs.0p25.2018010100.f006.grib2.923a8.idx' incompatible with GRIB file



*** PROFILER RESULTS ***
subset_dataset (/tmp/ipykernel_9729/286801345.py:2)
function called 1 times

         38012 function calls (36686 primitive calls) in 0.127 seconds

   Ordered by: cumulative time, internal time, call count
   List reduced from 674 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.127    0.127 286801345.py:2(subset_dataset)
        3    0.000    0.000    0.098    0.033 computation.py:802(apply_ufunc)
        3    0.000    0.000    0.094    0.031 computation.py:256(apply_dataarray_vfunc)
        1    0.000    0.000    0.067    0.067 common.py:1177(where)
  425/211    0.002    0.000    0.057    0.000 {built-in method numpy.asarray}
       95    0.000    0.000    0.057    0.001 variable.py:332(data)
       34    0.000    0.000    0.055    0.002 variable.py:507(values)
       34    0.000    0.000    0.055    0.002 variable.py:236(_as_array_or_item)
       29    0.000    0.000    

Dropdown(description='data file', options=('/local/train/GFS/gfs.0p25.2018010106.f006.grib2',), value='/local/…

Ignoring index file '/local/train/GFS/gfs.0p25.2018010106.f006.grib2.923a8.idx' incompatible with GRIB file


...

Ignoring index file '/local/train/GFS/gfs.0p25.2018011512.f006.grib2.923a8.idx' incompatible with GRIB file



*** PROFILER RESULTS ***
subset_dataset (/tmp/ipykernel_9729/286801345.py:2)
function called 1 times

         37583 function calls (36278 primitive calls) in 0.120 seconds

   Ordered by: cumulative time, internal time, call count
   List reduced from 624 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.120    0.120 286801345.py:2(subset_dataset)
        3    0.000    0.000    0.093    0.031 computation.py:802(apply_ufunc)
        3    0.000    0.000    0.089    0.030 computation.py:256(apply_dataarray_vfunc)
        1    0.000    0.000    0.062    0.062 common.py:1177(where)
  425/211    0.002    0.000    0.054    0.000 {built-in method numpy.asarray}
       95    0.000    0.000    0.054    0.001 variable.py:332(data)
        4    0.000    0.000    0.053    0.013 merge.py:319(merge_coordinates_without_align)
        4    0.000    0.000    0.053    0.013 merge.py:168(merge_collected)
       34    0

Dropdown(description='data file', options=('/local/train/GFS/gfs.0p25.2018011518.f006.grib2',), value='/local/…

Ignoring index file '/local/train/GFS/gfs.0p25.2018011518.f006.grib2.923a8.idx' incompatible with GRIB file



*** PROFILER RESULTS ***
subset_dataset (/tmp/ipykernel_9729/286801345.py:2)
function called 1 times

         37583 function calls (36278 primitive calls) in 0.122 seconds

   Ordered by: cumulative time, internal time, call count
   List reduced from 624 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.122    0.122 286801345.py:2(subset_dataset)
        3    0.000    0.000    0.094    0.031 computation.py:802(apply_ufunc)
        3    0.000    0.000    0.090    0.030 computation.py:256(apply_dataarray_vfunc)
        1    0.000    0.000    0.064    0.064 common.py:1177(where)
  425/211    0.002    0.000    0.055    0.000 {built-in method numpy.asarray}
       95    0.000    0.000    0.055    0.001 variable.py:332(data)
        4    0.000    0.000    0.054    0.014 merge.py:319(merge_coordinates_without_align)
        4    0.000    0.000    0.053    0.013 merge.py:168(merge_collected)
       34    0

 #### Now to download the files

In [22]:
#TODOs: 
# Do this for each region and concatenate the 3 dataframes into one dataframe. (Do we want to add column with region labeled?). 
# Create strings for each possible filename (i.e. 01 through 31 for 01 through 12 months for 2018 to 2020 years). 
# Use Srishti's S3 bucket and add a test csv file to the bucket (so we don't have to store locally). 
# Pull file download, df creation, df to csv save to s3 (forecast time) and file deletion into one loop function (based on dates above). Quick exit if error bc date doesn't exist (i.e. 31).
# Make sure we can pass tuples or some combination for level and variable name into function so that we can quickly change variables included. 
# Add a timeit call to understand how long it takes to run end-to-end pipeline. 

In [52]:
out_parquet =  'gfs.0p25.' + '2018' + '01' + '01' + '.f006.parquet'
filepath = '../train/GFS/parquet/' + out_parquet
test_df = pd.read_parquet(filepath, engine='pyarrow')

In [56]:
pd.set_option('display.max_rows', 100)
test_df.to_csv('test.csv')

In [55]:
s3_write_client = boto3.client('s3')
my_array_data = io.BytesIO()
pickle.dump(test_df, my_array_data)
my_array_data.seek(0)
s3_write_client.upload_fileobj(my_array_data, 'pollution-prediction','gfs_test.pkl')

In [62]:
pickle.loads(my_array_data)

TypeError: a bytes-like object is required, not '_io.BytesIO'